In [1]:
"""
learn from Paper named:
Solving two-stage robust optimization problems using a column-and-constraint generation method
Author:Su XY
Time:2019-3-1
Place of creation:iPso
Revised by:CAI Yun
Time:2022-4-30
"""

'\nlearn from Paper named:\nSolving two-stage robust optimization problems using a column-and-constraint generation method\nAuthor:Su XY\nTime:2019-3-1\nPlace of creation:iPso\nRevised by:CAI Yun\nTime:2022-4-30\n'

In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

ModuleNotFoundError: No module named 'gurobipy'

In [ ]:
#ref: Solving two-stage robust optimization problems using a column-and-constraint generation method

In [ ]:
#define master problem

In [ ]:

# Constant creation
f = [400, 414, 326]
a = [18, 25, 20]
C = [[22, 33, 24],
      [33, 23, 30],
      [20, 25, 27], ]
D = [206 + 40, 274 + 40, 220 + 40]
dl = [206, 274, 220]
du = [40, 40, 40]
k = 0  # Iterative counting
print('k: '+str(k))
# Create model
MP = gp.Model()  # Master-problem
SP = gp.Model()  # Sub-problem(KKT)
SPSD = gp.Model()  # Sub-problem (strong duality)
# Construction of Master-problem
# addVars
y = MP.addVars(len(f), lb=0, ub=1, obj=f, vtype=GRB.INTEGER, name='y')
z = MP.addVars(len(a), lb=0, obj=a, vtype=GRB.INTEGER, name='z')
#g = MP.addVars(3, lb=0, ub=1.0, name='g')
#deterministic problem
g = MP.addVars(3, lb=0, ub=0.0, name='g')
η = MP.addVar(obj=1.0, name='η')

# addConstrs
Column1 = MP.addConstrs((z[i] <= 800 * y[i] for i in range(3)), name='fixCap')
#Column4 = MP.addConstr(gp.quicksum(z[i] for i in range(3)) >= 772, name='zCap') #772=206+274+220+40*1.8
#deterministic problem
Column4 = MP.addConstr(gp.quicksum(z[i] for i in range(3)) >= 700, name='zCap') #772=206+274+220+40*1.8
Column5 = MP.addConstr(gp.quicksum(g[i] for i in range(2)) <= 1.2, name='uncertSet1')
Column6 = MP.addConstr(gp.quicksum(g[i] for i in range(3)) <= 1.8, name='uncertSet2')

#MP.Params.timeLimit = 100.0 # 等价于 MP.setParam('timeLimit', 100)
#MP.Params.InfUnbdInfo = 1  # Determines whether simplex (and crossover) will compute additional information when a model is determined to be infeasible or unbounded
#设置是否在控制台显示优化信息：
MP.Params.LogToConsole = 0
#设置线性规划的求解方法：
#MP.params.Method = 1  # 使用对偶单纯形法
MP.params.Method = 0  # 使用原始单纯形法(迭代慢，但占内存小）
#MP.params.Method = 2  # 使用内点法（gurobi称作barrier法）
#检查约束条件，模型得不到可行解时，才能用这个函数
#使用 MP.computeIIS() 检查不可行的约束条件
#MP.computeIIS()
#MP.write("MPilp.ilp")
#使用 m.feasRelax() 通过松弛最少的不可行约束条件，得到一个可行解（模型得不到可行解时，才能用这个函数）
if MP.Status == 3: # model is infeasible 
    print("Model is infeasible")
    MP.computeIIS()
    MP.write("model.ilp")
#print(MP.getConstrs())
MP.write("MPbd.lp")  # model print and visual inspection model,can open it with Notepad++
MP.optimize()  # Solve Model

print('Optimal value '+str(MP.ObjVal))
print('Optimal solution', end = " ")
for i in MP.getVars():
    print('%s = %g' % (i.varName, i.x), end = " ")
print()

LB = MP.objval  # get optimum value of model
print('LB: '+str(LB))

In [ ]:
#define Sub-problem by Benders-dual method (SP1)
#the resulting problem in (SP1) is a bilinear optimization problem

In [ ]:

d = [dl[i] + du[i] * g[i].x for i in range(3)]
# zupd = []
# zupd.append([z[i].x for i in range(3)])
SPSD = gp.Model("SPSD")
# Construction of Sub-problem (strong duality)
# addVars
lab = SPSD.addVars(3, lb=0, name='lambda')
w = SPSD.addVars(3, lb=0, name='w')
pi = SPSD.addVars(3, lb=0, name='pi')
#g = MP.addVars(3, lb=0, ub=1.0, name='g')
#deterministic problem
g = SPSD.addVars(3, lb=0, ub=0.0, name='g')
M = np.zeros((3))
for i in range(3):
    for j in range(3):
        M[i] = max(C[i][0], C[i][1], C[i][2])
# addConstrs
SD1 = SPSD.addConstrs(((lab[j] - pi[i]) <= C[i][j] for i in range(3) for j in range(3)), name='dualConstr')
SD2 = SPSD.addConstr(gp.quicksum(g[i] for i in range(2)) <= 1.2, name='uncertSet1')
SD3 = SPSD.addConstr(gp.quicksum(g[i] for i in range(3)) <= 1.8, name='uncertSet2')
SD4 = SPSD.addConstrs((w[j] <= lab[j] for j in range(3)), name='uncertSetLinear1') #SDSP4~6: product-linearization
SD5 = SPSD.addConstrs((w[j] <= g[j] * M[j] for j in range(3)), name='uncertSetLinear2')
SD6 = SPSD.addConstrs((w[j] >= (lab[j] - (1 - g[j]) * M[j]) for j in range(3)), name='uncertSetLinear3')
obj = gp.quicksum(dl[i] * lab[i] + du[i] * w[i] - z[i].x * pi[i] for i in range(3))
SPSD.Params.LogToConsole = 0
SPSD.setObjective(obj, GRB.MAXIMIZE)  # set Objective is max
SPSD.write("SPSD.lp")
SPSD.optimize()  # Solve Model

print('Optimal value '+str(SPSD.ObjVal))
print('Optimal solution', end = " ")
for i in SPSD.getVars():
    print('%s = %g' % (i.varName, i.x), end = " ")
print()

UB = LB - η.x + SPSD.objval
print('UB: '+str(UB))

In [ ]:
#given y and z, derive the optimal g, lambda and pi thus, a cutting plane is created

while UB - LB > 10e-4 and k < 8:
    k = k + 1  # Iterative counting
    print('k: '+str(k))
    MP.Params.LogToConsole = 0
    SPSD.Params.LogToConsole = 0
    # create varibales and add the following constraints
    #xx = MP.addVars(3, 3, lb=0, vtype=GRB.CONTINUOUS, name='x')
    #d = [dl[i] + du[i] * g[i].x for i in range(3)]
    #Column2 = MP.addConstrs(((gp.quicksum(xx[i, j] for j in range(3))) <= z[i] for i in range(3)), name='column2')
    #Column3 = MP.addConstrs(((gp.quicksum(xx[i, j] for i in range(3))) >= d[j] for j in range(3)), name='column3')
    #Column7 = MP.addConstr(gp.quicksum(C[i][j] * xx[i, j] for i in range(3) for j in range(3)) <= η)
    Column2 = MP.addConstr(gp.quicksum(dl[i] * lab[i].x + du[i] * w[i].x - z[i] * pi[i].x for i in range(3)) <= η , name='constrGen')
    MP.write('MPSD_' + str(k) + '.lp')
    MP.optimize()
    print('Optimal value '+str(MP.ObjVal))
    print('Optimal solution', end = " ")
    for i in MP.getVars():
        print('%s = %g' % (i.varName, i.x), end = " ")
    print()
    LB = MP.objval  # update LB
    print('LB: '+str(LB))
    obj = gp.quicksum(dl[i] * lab[i] + du[i] * w[i] - z[i].x * pi[i] for i in range(3))  # update SPSD's obj

    SPSD.setObjective(obj, GRB.MAXIMIZE)
    SPSD.write('SPSD_' + str(k) + '.lp')
    SPSD.optimize()
    print('Optimal value '+str(SPSD.ObjVal))
    print('Optimal solution', end = " ")
    for i in SPSD.getVars():
        print('%s = %g' % (i.varName, i.x), end = " ")
    print()
    UB = min(UB, LB - η.x + SPSD.objval)  # update UB
    print('UB: '+str(UB))
print(UB)
